# Introduction to Dask

In [ ]:
!pip install --upgrade dask distributed bokeh adlfs fsspec fastparquet pyarrow python-snappy lz4

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
from dask.distributed import Client

c = Client()
c

## Read data

Edit storage options per https://github.com/dask/adlfs.

In [ ]:
container_name = "isdweatherdatacontainer"

storage_options = {"account_name": "azureopendatastorage"}

In [ ]:
from adlfs import AzureBlobFileSystem

fs = AzureBlobFileSystem(**storage_options)
fs

In [ ]:
%%time
fs.ls(f"{container_name}/ISDWeather/year=2020")

In [ ]:
files = fs.glob(f"{container_name}/ISDWeather/year=2020/month=2/*.parquet")
files = [f"az://{file}" for file in files]
files[-5:]

In [ ]:
import dask.dataframe as dd

In [ ]:
%%time
ddf = dd.read_parquet(files, storage_options=storage_options, chunksize="20MB")
ddf

In [ ]:
ddf = ddf.persist()

In [ ]:
%%time
len(ddf)

In [ ]:
%%time
len(ddf)

In [ ]:
%%time
ddf.describe().compute()

In [ ]:
%%time
gbs = round(ddf.memory_usage(index=True, deep=True).sum().compute() / 1e9, 2)
print(f"ddf is {gbs} GBs")